# Examples on Nested Logit Model
Author: Tianyu Du
Examples here are modified from [Exercise 2: Nested logit model by Kenneth Train and Yves Croissant](https://cran.r-project.org/web/packages/mlogit/vignettes/e2nlogit.html)
The data set HC from mlogit contains data in R format on the choice of heating and central cooling system for 250 single-family, newly built houses in California.

The alternatives are:

- Gas central heat with cooling gcc,
- Electric central resistence heat with cooling ecc,
- Electric room resistence heat with cooling erc,
- Electric heat pump, which provides cooling also hpc,
- Gas central heat without cooling gc,
- Electric central resistence heat without cooling ec,
- Electric room resistence heat without cooling er.
- Heat pumps necessarily provide both heating and cooling such that heat pump without cooling is not an alternative.

The variables are:

- depvar gives the name of the chosen alternative,
- ich.alt are the installation cost for the heating portion of the system,
- icca is the installation cost for cooling
- och.alt are the operating cost for the heating portion of the system
- occa is the operating cost for cooling
- income is the annual income of the household

Note that the full installation cost of alternative gcc is ich.gcc+icca, and similarly for the operating cost and for the other alternatives with cooling.

## Load Essential Packages.

In [1]:
import argparse

import pandas as pd
import torch

from deepchoice.data import ChoiceDataset, JointDataset, utils
from deepchoice.model.nested_logit_model import NestedLogitModel
from deepchoice.utils.std import parameter_std

if torch.cuda.is_available():
    print(f'CUDA device used: {torch.cuda.get_device_name()}')

CUDA device used: NVIDIA GeForce RTX 3090


## Encompass Configurations in a Name Space Object

In [2]:
args = argparse.Namespace(data_path='./',
                          batch_size=-1,  # full-batch.
                          shuffle=False,
                          num_epochs=30000,
                          device='cuda' if torch.cuda.is_available() else 'cpu')

## Load Datasets

In [3]:
df = pd.read_csv('./HC.csv', index_col=0)
df = df.reset_index(drop=True)
df.head()

,depvar,icca,occa,income,ich,och,idx.id1,idx.id2,inc.room,inc.cooling,int.cooling,cooling.modes,room.modes
0,False,0.00,0.00,20,24.50,4.09,1,ec,0,0,0,False,False
1,False,27.28,2.95,20,7.86,4.09,1,ecc,0,20,1,True,False
2,False,0.00,0.00,20,7.37,3.85,1,er,20,0,0,False,True
3,True,27.28,2.95,20,8.79,3.85,1,erc,20,20,1,True,True
4,False,0.00,0.00,20,24.08,2.26,1,gc,0,0,0,False,False


In [4]:
# label
# what was actually chosen.
label = df[df['depvar'] == True].sort_values(by='idx.id1')['idx.id2'].reset_index(drop=True)
item_names = ['ec', 'ecc', 'er', 'erc', 'gc', 'gcc', 'hpc']
num_items = df['idx.id2'].nunique()
# cardinal encoder.
encoder = dict(zip(item_names, range(num_items)))
label = label.map(lambda x: encoder[x])
label = torch.LongTensor(label)

In [5]:
# category feature: no category feature, all features are item-level.
category_dataset = ChoiceDataset(label=label.clone()).to(args.device)

# item feature.
item_feat_cols = ['ich', 'och', 'icca', 'occa', 'inc.room', 'inc.cooling', 'int.cooling']
price_obs = utils.pivot3d(df, dim0='idx.id1', dim1='idx.id2', values=item_feat_cols)
item_dataset = ChoiceDataset(label=label, price_obs=price_obs).to(args.device)

# combine dataets
dataset = JointDataset(category=category_dataset, item=item_dataset)
data_loader = utils.create_data_loader(dataset)

# Example 1
Run a nested logit model on the data for two nests and one log-sum coefficient that applies to both nests. Note that the model is specified to have the cooling alternatives `{gcc, ecc, erc, hpc}` in one nest and the non-cooling alternatives `{gc, ec, er}` in another nest.

In [6]:
category_to_item = {0: ['gcc', 'ecc', 'erc', 'hpc'],
                    1: ['gc', 'ec', 'er']}

# convert names 
for k, v in category_to_item.items():
    v = [encoder[item] for item in v]
    category_to_item[k] = sorted(v)

model = NestedLogitModel(category_to_item=category_to_item,
                         category_coef_variation_dict={},
                         category_num_param_dict={},
                         item_coef_variation_dict={'price_obs': 'constant'},
                         item_num_param_dict={'price_obs': 7},
                         shared_lambda=True)

model = model.to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)

for e in range(args.num_epochs):
    for batch in data_loader:
        loss = model.negative_log_likelihood(batch, batch['item'].label)
        if e % (args.num_epochs // 10) == 0:
            print(f'{e=:}: {loss=:}, {model.lambda_weight.item()=:}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


e=0: loss=29266.6640625, model.lambda_weight.item()=1.0
e=3000: loss=180.41995239257812, model.lambda_weight.item()=0.9177842140197754
e=6000: loss=178.1299591064453, model.lambda_weight.item()=0.5788174271583557
e=9000: loss=178.12496948242188, model.lambda_weight.item()=0.5862539410591125
e=12000: loss=178.125, model.lambda_weight.item()=0.5866018533706665
e=15000: loss=178.19720458984375, model.lambda_weight.item()=0.5874289870262146
e=18000: loss=178.12911987304688, model.lambda_weight.item()=0.5867705345153809
e=21000: loss=178.12567138671875, model.lambda_weight.item()=0.5864641666412354
e=24000: loss=178.127197265625, model.lambda_weight.item()=0.586590588092804
e=27000: loss=178.12474060058594, model.lambda_weight.item()=0.5859836339950562


## R Output
```
## 
## Call:
## mlogit(formula = depvar ~ ich + och + icca + occa + inc.room + 
##     inc.cooling + int.cooling | 0, data = HC, nests = list(cooling = c("gcc", 
##     "ecc", "erc", "hpc"), other = c("gc", "ec", "er")), un.nest.el = TRUE)
## 
## Frequencies of alternatives:choice
##    ec   ecc    er   erc    gc   gcc   hpc 
## 0.004 0.016 0.032 0.004 0.096 0.744 0.104 
## 
## bfgs method
## 11 iterations, 0h:0m:0s 
## g'(-H)^-1g = 7.26E-06 
## successive function values within tolerance limits 
## 
## Coefficients :
##              Estimate Std. Error z-value  Pr(>|z|)    
## ich         -0.554878   0.144205 -3.8478 0.0001192 ***
## och         -0.857886   0.255313 -3.3601 0.0007791 ***
## icca        -0.225079   0.144423 -1.5585 0.1191212    
## occa        -1.089458   1.219821 -0.8931 0.3717882    
## inc.room    -0.378971   0.099631 -3.8038 0.0001425 ***
## inc.cooling  0.249575   0.059213  4.2149 2.499e-05 ***
## int.cooling -6.000415   5.562423 -1.0787 0.2807030    
## iv           0.585922   0.179708  3.2604 0.0011125 ** 
## ---
## Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
## 
## Log-Likelihood: -178.12
```

In [7]:
for k, v in model.named_parameters():
    print(f'{k} = {v.detach().cpu()}')

lambda_weight = tensor([0.5867])
item_coef_dict.price_obs.coef = tensor([-0.5591, -0.8631, -0.2248, -1.0893, -0.3824,  0.2521, -6.0042])


## Computing Standard Errors
**NOTE**: We are computing the standard errors using $\sqrt{\text{diag}(H^{-1})}$, where $H$ is the
hessian of negative log-likelihood with repsect to model parameters. This leads to slight different
results compared with R implementation.

In [8]:
batch = dataset[torch.arange(len(dataset))]
def nll_loss(model):
    return model.negative_log_likelihood(batch, batch['item'].label)

std = parameter_std(model, nll_loss)
std

{'lambda_weight': tensor([0.1677]),
 'item_coef_dict.price_obs.coef': tensor([0.1475, 0.2413, 0.1124, 1.0530, 0.1030, 0.0536, 4.9208])}

# Example 2
Re-estimate the model with the room alternatives in one nest and the central alternatives in another nest. (Note that a heat pump is a central system.)

In [9]:
category_to_item = {0: ['ec', 'ecc', 'gc', 'gcc', 'hpc'],
                    1: ['er', 'erc']}
for k, v in category_to_item.items():
    v = [encoder[item] for item in v]
    category_to_item[k] = sorted(v)

model = NestedLogitModel(category_to_item=category_to_item,
                            category_coef_variation_dict={},
                            category_num_param_dict={},
                        #  category_num_param_dict={'intercept': 1},
                            item_coef_variation_dict={'price_obs': 'constant'},
                            item_num_param_dict={'price_obs': 7},
                            shared_lambda=True
                            )

model = model.to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.9999)

for e in range(args.num_epochs):
    for batch in data_loader:
        loss = model.negative_log_likelihood(batch, batch['item'].label)
        if e % (args.num_epochs // 10) == 0:
            print(f'{e=:}: {loss=:}, {model.lambda_weight.item()=:}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model.clamp_lambdas()
    scheduler.step()

e=0: loss=13013.70703125, model.lambda_weight.item()=1.0
e=3000: loss=180.76327514648438, model.lambda_weight.item()=1.3446736335754395
e=6000: loss=180.1411590576172, model.lambda_weight.item()=1.2529666423797607
e=9000: loss=180.02523803710938, model.lambda_weight.item()=1.336637258529663
e=12000: loss=180.0230712890625, model.lambda_weight.item()=1.3618589639663696
e=15000: loss=180.02305603027344, model.lambda_weight.item()=1.362200140953064
e=18000: loss=180.0230712890625, model.lambda_weight.item()=1.3621675968170166
e=21000: loss=180.0230712890625, model.lambda_weight.item()=1.3621395826339722
e=24000: loss=180.02325439453125, model.lambda_weight.item()=1.3622841835021973
e=27000: loss=180.0230712890625, model.lambda_weight.item()=1.362113118171692


## R Output
```
## 
## Call:
## mlogit(formula = depvar ~ ich + och + icca + occa + inc.room + 
##     inc.cooling + int.cooling | 0, data = HC, nests = list(central = c("ec", 
##     "ecc", "gc", "gcc", "hpc"), room = c("er", "erc")), un.nest.el = TRUE)
## 
## Frequencies of alternatives:choice
##    ec   ecc    er   erc    gc   gcc   hpc 
## 0.004 0.016 0.032 0.004 0.096 0.744 0.104 
## 
## bfgs method
## 10 iterations, 0h:0m:0s 
## g'(-H)^-1g = 5.87E-07 
## gradient close to zero 
## 
## Coefficients :
##              Estimate Std. Error z-value Pr(>|z|)  
## ich          -1.13818    0.54216 -2.0993  0.03579 *
## och          -1.82532    0.93228 -1.9579  0.05024 .
## icca         -0.33746    0.26934 -1.2529  0.21024  
## occa         -2.06328    1.89726 -1.0875  0.27681  
## inc.room     -0.75722    0.34292 -2.2081  0.02723 *
## inc.cooling   0.41689    0.20742  2.0099  0.04444 *
## int.cooling -13.82487    7.94031 -1.7411  0.08167 .
## iv            1.36201    0.65393  2.0828  0.03727 *
## ---
## Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
## 
## Log-Likelihood: -180.02

```

In [10]:
for k, v in model.named_parameters():
    print(f'{k} = {v.detach().cpu()}')

lambda_weight = tensor([1.3622])
item_coef_dict.price_obs.coef = tensor([ -1.1383,  -1.8255,  -0.3374,  -2.0634,  -0.7573,   0.4170, -13.8256])


## Computing Standard Errors
**NOTE**: We are computing the standard errors using $\sqrt{\text{diag}(H^{-1})}$, where $H$ is the
hessian of negative log-likelihood with repsect to model parameters. This leads to slight different
results compared with R implementation.

In [11]:
batch = dataset[torch.arange(len(dataset))]
def nll_loss(model):
    return model.negative_log_likelihood(batch, batch['item'].label)

std = parameter_std(model, nll_loss)
std

{'lambda_weight': tensor([0.5553]),
 'item_coef_dict.price_obs.coef': tensor([0.4444, 0.7384, 0.2026, 1.7623, 0.2785, 0.1701, 8.0962])}

# Example 3
Rewrite the code to allow three nests. For simplicity, estimate only one log-sum coefficient which is applied to all three nests. Estimate a model with alternatives gcc, ecc and erc in a nest, hpc in a nest alone, and alternatives gc, ec and er in a nest. Does this model seem better or worse than the model in exercise 1, which puts alternative hpc in the same nest as alternatives gcc, ecc and erc?

In [12]:
category_to_item = {0: ['gcc', 'ecc', 'erc'],
                    1: ['hpc'],
                    2: ['gc', 'ec', 'er']}
for k, v in category_to_item.items():
    v = [encoder[item] for item in v]
    category_to_item[k] = sorted(v)

model = NestedLogitModel(category_to_item=category_to_item,
                         category_coef_variation_dict={},
                         category_num_param_dict={},
                         item_coef_variation_dict={'price_obs': 'constant'},
                         item_num_param_dict={'price_obs': 7},
                         shared_lambda=True
                         )

model = model.to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.9999)

for e in range(args.num_epochs):
    for batch in data_loader:
        loss = model.negative_log_likelihood(batch, batch['item'].label)
        if e % (args.num_epochs // 10) == 0:
            print(f'{e=:}: {loss=:}, {model.lambda_weight=:}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # model.clamp_lambdas()
    scheduler.step()

e=0: loss=2655.865478515625, model.lambda_weight=Parameter containing:
tensor([1.], device='cuda:0', requires_grad=True)
e=3000: loss=194.99459838867188, model.lambda_weight=Parameter containing:
tensor([1.0698], device='cuda:0', requires_grad=True)
e=6000: loss=182.6705322265625, model.lambda_weight=Parameter containing:
tensor([0.8543], device='cuda:0', requires_grad=True)
e=9000: loss=181.5911102294922, model.lambda_weight=Parameter containing:
tensor([0.9128], device='cuda:0', requires_grad=True)
e=12000: loss=180.92767333984375, model.lambda_weight=Parameter containing:
tensor([0.9212], device='cuda:0', requires_grad=True)
e=15000: loss=180.55140686035156, model.lambda_weight=Parameter containing:
tensor([0.9310], device='cuda:0', requires_grad=True)
e=18000: loss=180.37588500976562, model.lambda_weight=Parameter containing:
tensor([0.9398], device='cuda:0', requires_grad=True)
e=21000: loss=180.30047607421875, model.lambda_weight=Parameter containing:
tensor([0.9466], device='cud

## R Output
```
## 
## Call:
## mlogit(formula = depvar ~ ich + och + icca + occa + inc.room + 
##     inc.cooling + int.cooling | 0, data = HC, nests = list(n1 = c("gcc", 
##     "ecc", "erc"), n2 = c("hpc"), n3 = c("gc", "ec", "er")), 
##     un.nest.el = TRUE)
## 
## Frequencies of alternatives:choice
##    ec   ecc    er   erc    gc   gcc   hpc 
## 0.004 0.016 0.032 0.004 0.096 0.744 0.104 
## 
## bfgs method
## 8 iterations, 0h:0m:0s 
## g'(-H)^-1g = 3.71E-08 
## gradient close to zero 
## 
## Coefficients :
##               Estimate Std. Error z-value  Pr(>|z|)    
## ich          -0.838394   0.100546 -8.3384 < 2.2e-16 ***
## och          -1.331598   0.252069 -5.2827 1.273e-07 ***
## icca         -0.256131   0.145564 -1.7596   0.07848 .  
## occa         -1.405656   1.207281 -1.1643   0.24430    
## inc.room     -0.571352   0.077950 -7.3297 2.307e-13 ***
## inc.cooling   0.311355   0.056357  5.5247 3.301e-08 ***
## int.cooling -10.413384   5.612445 -1.8554   0.06354 .  
## iv            0.956544   0.180722  5.2929 1.204e-07 ***
## ---
## Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
## 
## Log-Likelihood: -180.26
```

In [13]:
for k, v in model.named_parameters():
    print(f'{k} = {v.detach().cpu()}')

lambda_weight = tensor([0.9563])
item_coef_dict.price_obs.coef = tensor([ -0.8382,  -1.3314,  -0.2567,  -1.4107,  -0.5712,   0.3113, -10.3844])


## Computing Standard Errors
**NOTE**: We are computing the standard errors using $\sqrt{\text{diag}(H^{-1})}$, where $H$ is the
hessian of negative log-likelihood with repsect to model parameters. This leads to slight different
results compared with R implementation.

In [14]:
batch = dataset[torch.arange(len(dataset))]
def nll_loss(model):
    return model.negative_log_likelihood(batch, batch['item'].label)

std = parameter_std(model, nll_loss)
std

{'lambda_weight': tensor([0.1970]),
 'item_coef_dict.price_obs.coef': tensor([0.0991, 0.1849, 0.1263, 1.1446, 0.0750, 0.0551, 5.1917])}